In [1]:
from imputationLibrary import forwardFilling, hotDeck, meanImputation, movingAverage, splineInterpolation, separate, randomSampleImputation
from imputationLibrary.util import util
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from tsmoothie.smoother import *
import matplotlib.pyplot as plt

In [2]:
for i in range(0,30):
    df = pd.read_csv(r'output\/CompleteCovid.csv',index_col=0, parse_dates=True)
    df_train = df.loc['2020-01-22 09:00:00':'2020-02-26 02:00:00'].dropna(how='all', axis=1) #Selecting training data
    df_test = df.loc['2020-02-26 02:30:00':, df_train.columns] #Selecting test data
    period=30
    df_white_noise, df_seasonal, df_trended, df_trend_and_seasonal = separate.separate(df_train, period)
    df_seasonal_train, df_seasonal_test = splineInterpolation.input(df_train.loc[:, df_seasonal.columns], df_test.loc[:, df_seasonal.columns])
    df_white_noise_train, df_white_noise_test =  meanImputation.input(df_train.loc[:, df_white_noise.columns], df_test.loc[:, df_white_noise.columns])
    df_trended_train, df_trended_test =  forwardFilling.input(df_train.loc[:, df_trended.columns], df_test.loc[:, df_trended.columns])
    df_trend_and_seasonal_train, df_trend_and_seasonal_test =  splineInterpolation.input(df_train.loc[:, df_trend_and_seasonal.columns], df_test.loc[:, df_trend_and_seasonal.columns])
    df_train = pd.concat([df_seasonal_train, df_white_noise_train, df_trended_train, df_trend_and_seasonal_train], axis=1, join='outer')
    df_test = pd.concat([df_seasonal_test, df_white_noise_test, df_trended_test, df_trend_and_seasonal_test], axis=1, join='outer')


    ### USE KALMAN FILTER TO SMOOTH ALL DATA (ONLY VISUALIZATION PURPOSE) ###

    smoother_train = KalmanSmoother(component='level_longseason', 
                              component_noise={'level':0.1, 'longseason':0.1}, 
                              n_longseasons=365)
    smoother_train.smooth(df_train.T)

    smoother_test = KalmanSmoother(component='level_longseason', 
                              component_noise={'level':0.1, 'longseason':0.1}, 
                              n_longseasons=365)
    smoother_test.smooth(df_test.T)

    df_train = pd.DataFrame(data = smoother_train.smooth_data.T, index = df_train.index, columns= df_train.columns)
    df_test = pd.DataFrame(data = smoother_test.smooth_data.T, index = df_test.index, columns= df_test.columns)

    df_train.to_csv(r'output\/CompleteCovidNatureKalmanTrain_'+str(i)+'.csv', index = True)
    df_test.to_csv(r'output\/CompleteCovidNatureKalmanTest_'+str(i)+'.csv', index = True)

In [3]:
df_white_noise.columns

Index(['Asia_confirmed_cases', 'Asia_deaths', 'Asia_recovered',
       'Europe_confirmed_cases', 'Europe_deaths', 'Europe_recovered',
       'Americas_confirmed_cases', 'Americas_deaths', 'Americas_recovered',
       'Oceania_confirmed_cases', 'Oceania_deaths', 'Oceania_recovered',
       'Africa_confirmed_cases', 'Africa_deaths', 'Africa_recovered'],
      dtype='object')

In [4]:
df_seasonal.columns

Index([], dtype='object')

In [5]:
df_trended.columns

Index([], dtype='object')

In [6]:
df_trend_and_seasonal.columns

Index([], dtype='object')

In [7]:
df_train

,Asia_confirmed_cases,Asia_deaths,Asia_recovered,Europe_confirmed_cases,Europe_deaths,Europe_recovered,Americas_confirmed_cases,Americas_deaths,Americas_recovered,Oceania_confirmed_cases,Oceania_deaths,Oceania_recovered,Africa_confirmed_cases,Africa_deaths,Africa_recovered
update_time,,,,,,,,,,,,,,,
2020-01-22 09:00:00,1522.133761,40.244050,198.789977,30.110052,0.688709,2.341612,6.109290,0.0,0.270521,6.592666,0.0,0.525053,0.991659,0.0,0.0
2020-01-22 09:30:00,1786.566110,47.468829,240.598998,30.202735,0.690827,2.348811,6.126829,0.0,0.271291,6.612743,0.0,0.526662,0.994733,0.0,0.0
2020-01-22 10:00:00,1956.168040,52.102997,267.413970,30.262182,0.692185,2.353429,6.137598,0.0,0.271759,6.625620,0.0,0.527694,0.996705,0.0,0.0
2020-01-22 10:30:00,2064.785328,55.071347,284.586043,30.300287,0.693056,2.356389,6.143752,0.0,0.272020,6.633874,0.0,0.528355,0.997969,0.0,0.0
2020-01-22 11:00:00,2134.094821,56.966275,295.542213,30.324692,0.693613,2.358285,6.146528,0.0,0.272125,6.639160,0.0,0.528779,0.998778,0.0,0.0
2020-01-22 11:30:00,2177.929856,58.165986,302.469153,30.340300,0.693970,2.359497,6.146485,0.0,0.272097,6.642540,0.0,0.529050,0.999296,0.0,0.0
2020-01-22 12:00:00,2205.040570,58.909959,306.749537,30.350260,0.694197,2.360271,6.143622,0.0,0.271929,6.644697,0.0,0.529222,0.999626,0.0,0.0
2020-01-22 12:30:00,2220.839915,59.346736,309.237945,30.356595,0.694342,2.360763,6.137374,0.0,0.271588,6.646069,0.0,0.529332,0.999836,0.0,0.0
2020-01-22 13:00:00,2228.483758,59.563559,310.431382,30.360603,0.694434,2.361074,6.126503,0.0,0.271006,6.646936,0.0,0.529402,0.999969,0.0,0.0


In [8]:
df_test

,Asia_confirmed_cases,Asia_deaths,Asia_recovered,Europe_confirmed_cases,Europe_deaths,Europe_recovered,Americas_confirmed_cases,Americas_deaths,Americas_recovered,Oceania_confirmed_cases,Oceania_deaths,Oceania_recovered,Africa_confirmed_cases,Africa_deaths,Africa_recovered
update_time,,,,,,,,,,,,,,,
2020-02-26 02:30:00,1804.242243,47.592917,259.907034,30.265081,0.693180,2.346948,6.108787,0.000573,0.270176,6.590639,1.941986e-05,0.525408,0.992150,7.878890e-07,0.000197
2020-02-26 03:00:00,1733.877999,45.684219,255.309907,30.298334,0.693591,2.351951,6.127173,0.000370,0.271101,6.611409,1.205775e-05,0.526882,0.995049,3.591214e-07,0.000123
2020-02-26 03:30:00,1558.354273,40.835107,238.742552,30.319581,0.693852,2.355156,6.138964,0.000239,0.271693,6.624731,7.326838e-06,0.527827,0.996908,8.210728e-08,0.000075
2020-02-26 04:00:00,1680.471090,44.159261,256.998312,30.333117,0.694017,2.357205,6.146520,0.000155,0.272073,6.633270,4.285216e-06,0.528433,0.998100,-9.749480e-08,0.000044
2020-02-26 04:30:00,1687.043224,44.246242,263.653779,30.341702,0.694121,2.358513,6.151359,0.000101,0.272317,6.638738,2.328117e-06,0.528821,0.998863,-2.145754e-07,0.000025
2020-02-26 05:00:00,1579.386781,41.113556,260.038498,30.347104,0.694184,2.359346,6.154453,0.000067,0.272472,6.642235,1.067242e-06,0.529069,0.999351,-2.915366e-07,0.000012
2020-02-26 05:30:00,1752.983350,45.810175,279.701436,30.350460,0.694223,2.359873,6.156430,0.000045,0.272571,6.644467,2.532768e-07,0.529227,0.999662,-3.427617e-07,0.000004
2020-02-26 06:00:00,1825.767622,47.606468,292.319273,30.352497,0.694245,2.360204,6.157691,0.000031,0.272634,6.645886,-2.738560e-07,0.529327,0.999860,-3.774858e-07,-0.000002
2020-02-26 06:30:00,1812.267636,46.861015,300.411272,30.353678,0.694257,2.360410,6.158497,0.000022,0.272673,6.646783,-6.169361e-07,0.529391,0.999985,-4.016381e-07,-0.000005
